In [ ]:
pip install fastf1

In [1]:
import fastf1 as ff1
from fastf1 import plotting
import numpy as np
import pandas as pd
import itertools
from itertools import product


In [3]:
    race = ff1.get_session(2020, 2, 'R')
    laps = race.load_laps(with_telemetry=True)



/Users/tianyangzhou/opt/anaconda3/lib/python3.9/site-packages/fastf1/core.py:1095: FutureWarning: `Session.load_laps` is deprecated and will beremoved in a future version.
Use `Session.load` instead.
  warnings.warn("`Session.load_laps` is deprecated and will be"
core           INFO 	Loading data for Styrian Grand Prix - Race [v2.2.4]
api            INFO 	Fetching driver list...
api            INFO 	Fetching timing data...
api            INFO 	Parsing timing data...
api            INFO 	Fetching timing app data...
core           INFO 	Processing timing data...
api            INFO 	Fetching session status data...
api            INFO 	Fetching track status data...
api            INFO 	Fetching car data...
api            INFO 	Parsing car data...
api            INFO 	Fetching position data...
api            INFO 	Parsing position data...
api         WARNING 	Driver 241: Position data is incomplete!
api         WARNING 	Driver 242: Position data is incomplete!
api         WARNING 	Driver 2

In [10]:
plotting.setup_mpl()
#ff1.Cache.enable_cache('')
# not important warnings & not relevant
pd.options.mode.chained_assignment = None 

In [11]:
def race(year, race_num):
    """
    This function fetches race information and 
    the performance of drivers of that race
    """
    summarized_distance_total_year = pd.DataFrame()
    lap_info_total_year= pd.DataFrame()
    race_info_total_year= pd.DataFrame()
    
    # Load the session data
    race = ff1.get_session(year, race_num, 'R')
    # Get the laps
    laps = race.load_laps(with_telemetry=True)

    #Get race info
    race_info= pd.DataFrame()
    race_info=race_info.append(race.results)
    race_info['Race'] = race_num
    race_info['year'] = year

    
    #Get lap info
    lap_info= pd.DataFrame()
    lap_info=race_info.append(laps)
    lap_info['Race'] = race_num
    lap_info['year'] = year

    
    race_info_total_year= race_info_total_year.append(race_info)
    lap_info_total_year= lap_info_total_year.append(lap_info)

    #Get driver list/pairs
    driver_list = laps['DriverNumber'].unique().tolist()
  
    
    #loop
    summarized_distance_total = pd.DataFrame()
    for pairs in itertools.product(driver_list, driver_list): # Get laps of the drivers
        # Get laps of the drivers (formerd and latterd)
        f=pairs[0]
        l=pairs[1]
        f_str="% s" % f
        l_str="% s" % l
        laps_former = laps.pick_driver(f)
        laps_latter = laps.pick_driver(l)
    #Since the race started in lap 2 because lap 1 was the warm up lap, we subtract 1.

        laps_former['RaceLapNumber'] = laps_former['LapNumber'] - 1
        laps_latter['RaceLapNumber'] = laps_latter['LapNumber'] - 1

        full_distance = pd.DataFrame()
        summarized_distance = pd.DataFrame()
        

    #Next, what we need to do is loop through all the laps one by one. 
    #The reason we do this is because if we don’t, the ‘Distance’ variable 
    #in the telemetry data will range from 0 meters until the entire length 
    #Since we want to compare lap-by-lap, the telemetry data needs to be requested again and again 
    #to reset the distance variable to 0.
        for lap in laps_latter.iterlaps():
            try:
                telemetry = lap[1].get_car_data().add_distance().add_driver_ahead()

            # Only run this loop when driver ahead is former
                telemetry = telemetry.loc[telemetry['DriverAhead'] == f_str]

                if len(telemetry) != 0:
                # Full distance
                    lap_telemetry = telemetry[['Distance', 'DistanceToDriverAhead']]
                    lap_telemetry.loc[:, 'Lap'] = lap[0] + 1

                    full_distance = full_distance.append(lap_telemetry)

                # Average distance
                    distance_mean = np.nanmean(telemetry['DistanceToDriverAhead'])


                #laps difference between a pair
                    summarized_distance = summarized_distance.append({
                        'Lap': lap[0] + 1,
                        'total_lap': len(laps_latter),
                        'position': int(race.results.loc[race.results.loc[:,'DriverNumber']==l_str,"Position"]), 
                        'Mean': distance_mean,
                        'Former': f,
                        'Latter': l, 
                        'race_num':race_num,
                        'year':year,
                    }, ignore_index = True)
            except:
                pass
        summarized_distance_total = summarized_distance_total.append(summarized_distance)        
      
    summarized_distance_total_year=summarized_distance_total_year.append(summarized_distance_total)
   
    return  summarized_distance_total_year,lap_info_total_year,race_info_total_year


    

In [12]:
#write for loop to extract datasets for 2021
totalraces=list(range(17,18))
summarized_distance = pd.DataFrame()
summarized_lap_info= pd.DataFrame()
summarized_race_info= pd.DataFrame()
for i in totalraces:
    try:
        distance,lapinfo,raceinfo=race(2021, i)
        summarized_race_info= summarized_race_info.append(raceinfo)
        summarized_lap_info= summarized_lap_info.append(lapinfo)
        summarized_distance = summarized_distance.append(distance)
    except:
        print("Something went wrong for race"+"i")

summarized_distance .to_csv(r"C:/Users/tz2156/Downloads/2021distance17_22csv", index=False)
summarized_lap_info.to_csv(r"C:/Users/tz2156/Downloads/2021lap17_22.csv", index=False)
summarized_race_info.to_csv(r"C:/Users/tz2156/Downloads/2021race17_22.csv", index=False)



/Users/tianyangzhou/opt/anaconda3/lib/python3.9/site-packages/fastf1/core.py:1095: FutureWarning: `Session.load_laps` is deprecated and will beremoved in a future version.
Use `Session.load` instead.
  warnings.warn("`Session.load_laps` is deprecated and will be"
core           INFO 	Loading data for United States Grand Prix - Race [v2.2.4]
api            INFO 	Fetching driver list...
api            INFO 	Fetching timing data...
api            INFO 	Parsing timing data...
api            INFO 	Fetching timing app data...
core           INFO 	Processing timing data...
api            INFO 	Fetching session status data...
api            INFO 	Fetching track status data...
api            INFO 	Fetching car data...
api            INFO 	Parsing car data...


Something went wrong for racei


FileNotFoundError: [Errno 2] No such file or directory: 'C:/Users/tz2156/Downloads/2021distance17_22csv'

In [8]:
race = ff1.get_session(2021, 14, 'R')
laps = race.load_laps(with_telemetry=True)


api         WARNING 	

NO CACHE! Api caching has not been enabled! 
	It is highly recommended to enable this feature for much faster data loading!
	Use `fastf1.Cache.enable_cache('path/to/cache/')`

/Users/tianyangzhou/opt/anaconda3/lib/python3.9/site-packages/fastf1/core.py:1095: FutureWarning: `Session.load_laps` is deprecated and will beremoved in a future version.
Use `Session.load` instead.
  warnings.warn("`Session.load_laps` is deprecated and will be"
core           INFO 	Loading data for Italian Grand Prix - Race [v2.2.4]
api            INFO 	Fetching driver list...
api            INFO 	Fetching timing data...
api            INFO 	Parsing timing data...
api            INFO 	Fetching timing app data...
core           INFO 	Processing timing data...
api            INFO 	Fetching session status data...
api            INFO 	Fetching track status data...
api            INFO 	Fetching car data...
api            INFO 	Parsing car data...
api            INFO 	Fetching position data...
